## Importing Dependencies

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
from tqdm.notebook import tqdm

In [3]:
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Technical Indicators Calculation

In [6]:
df = pd.read_csv('10_Year_Historical_Preprocessed.csv')

In [7]:
df = df[['Date', 'Ticker', 'Adj Close']]
df['Date'] = pd.to_datetime(df['Date'])

In [8]:

def exponential_moving_avg(df: pd.DataFrame, price_col='Adj Close', window_size=20):
    return df[price_col].ewm(span=window_size, adjust=False).mean()

def macd_line(df: pd.DataFrame, price_col='Adj Close', short_window=12, long_window=26):
    short_ema = exponential_moving_avg(df, price_col, short_window)
    long_ema = exponential_moving_avg(df, price_col, long_window)
    return short_ema - long_ema

def macd_signal(df: pd.DataFrame, price_col='Adj Close', signal_window=9, short_window=12, long_window=26):
    macd = macd_line(df, price_col, short_window, long_window)
    return macd.ewm(span=signal_window, adjust=False).mean()


In [9]:
df['EMA 20'] = df.groupby('Ticker').apply(lambda x: exponential_moving_avg(x, price_col='Adj Close', window_size=20)).reset_index(level=0, drop=True)
df['EMA 50'] = df.groupby('Ticker').apply(lambda x: exponential_moving_avg(x, price_col='Adj Close', window_size=50)).reset_index(level=0, drop=True)
df['MACD Line'] = df.groupby('Ticker').apply(lambda x: macd_line(x, price_col='Adj Close')).reset_index(level=0, drop=True)
df['MACD Signal'] = df.groupby('Ticker').apply(lambda x: macd_signal(x, price_col='Adj Close')).reset_index(level=0, drop=True)

## Feature Scaling

In [10]:
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].astype(float)

In [11]:
def scale_group(group):
    scaler = MinMaxScaler(feature_range=(0, 1))
    group.iloc[:, 2:] = scaler.fit_transform(group.iloc[:, 2:])
    return group

In [12]:
scaled_df = df.groupby('Ticker').apply(scale_group)

In [13]:
scaled_df = scaled_df.drop('Ticker', axis=1).reset_index().drop('level_1', axis=1)

## Saving scaled dataset

In [14]:
scaled_df.to_csv('10_Year_Historical_Scaled.csv')